In [1]:
data_var = '2024-01-07'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9664,2024-01-07,Espanha Acb,13:00,Barcelona,Obradoiro CAB,1.08,8.10,163.5,1.86,1.94,-15.5,2.08,1.68,jyfRSa5K,0.925926,0.123457,0.537634,0.515464,0.049383,0.4,0.6,NaN,NaN,106.056,17.189024,0.162075,275.982,114.465647,0.414758,105.100,265.676,109.06,195.30,0.0,0.0,0.0,0.0,1.081457,0.029773,0.150448,-0.29,-0.058,-1.379310,0.000000,0.0,0.000000,1.82,0.364,19.505495,0.000000,0.0,0.000000
9665,2024-01-07,Espanha Acb,13:00,Valencia,Gran Canaria,1.45,2.77,161.5,1.81,1.95,-5.5,1.98,1.82,CpeNTujE,0.689655,0.361011,0.552486,0.512821,0.050666,0.4,0.4,NaN,NaN,183.486,140.119288,0.763651,162.664,104.024845,0.639507,113.288,229.038,93.75,154.84,0.0,0.0,0.0,0.0,0.442361,0.052657,0.059546,0.01,0.002,225.000000,0.000000,0.0,0.000000,1.68,0.336,5.267857,0.000000,0.0,0.000000
9666,2024-01-07,Espanha Acb,14:30,Baskonia,Real Madrid,2.00,1.81,168.5,1.88,1.90,-1.5,2.08,1.68,b1gVRJKQ,0.500000,0.552486,0.531915,0.526316,0.052486,0.6,0.4,NaN,NaN,137.222,42.916789,0.312754,108.576,16.405825,0.151100,119.530,109.714,99.64,134.33,0.0,0.0,0.0,0.0,0.070525,0.007483,0.150448,0.25,0.050,20.000000,0.000000,0.0,0.000000,1.25,0.250,3.240000,0.000000,0.0,0.000000
9667,2024-01-07,Eua Nba,15:00,Cleveland Cavaliers,San Antonio Spurs,1.22,5.11,230.5,1.80,2.00,-11.5,2.00,1.71,tUxQqqgF,0.819672,0.195695,0.555556,0.500000,0.015367,0.4,0.4,NaN,NaN,230.850,73.663269,0.319096,497.160,167.915597,0.337750,203.668,392.732,136.80,686.00,0.0,1.0,0.0,0.0,0.869082,0.074432,0.110545,-1.15,-0.230,-0.956522,0.650477,0.7,0.049523,-2.54,-0.508,-8.090551,0.235472,0.1,-0.135472
9668,2024-01-07,Eua Nba,17:00,Brooklyn Nets,Portland Trail Blazers,1.32,3.93,225.5,1.86,2.07,-9.5,2.06,1.87,fXtUr38L,0.757576,0.254453,0.537634,0.483092,0.012029,0.8,0.6,NaN,NaN,299.956,93.666092,0.312266,421.506,219.565107,0.520906,261.894,482.402,357.12,565.47,1.0,1.0,0.0,0.0,0.703066,0.075569,0.068372,-0.84,-0.168,-1.904762,0.566119,0.6,0.033881,-5.00,-1.000,-2.930000,0.212951,0.2,-0.012951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9778,2024-01-07,Suíça Liga Sb,12:00,BBC Monthey,Regio Basel,1.06,7.80,161.5,1.85,1.85,-16.5,2.03,1.70,Y5Wv1e6p,0.943396,0.128205,0.540541,0.540541,0.071601,0.0,0.0,NaN,NaN,248.072,123.060716,0.496069,481.296,317.271268,0.659202,0.000,0.000,238.08,876.90,0.0,1.0,0.0,0.0,1.075824,0.000000,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9779,2024-01-07,Suíça Liga Sb,12:00,SAM Massagno,Vevey Riviera,1.36,2.97,162.5,1.80,1.86,-7.5,2.03,1.70,prF2Kq75,0.735294,0.336700,0.555556,0.537634,0.071994,0.0,0.0,NaN,NaN,99.300,9.703213,0.097716,158.224,72.917351,0.460849,0.000,0.000,106.08,132.30,0.0,0.0,0.0,0.0,0.525839,0.023184,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9780,2024-01-07,Suíça Liga Sb Feminina,12:00,Baden F,BBC Troistorrents F,7.60,1.06,141.5,1.80,1.87,14.5,2.00,1.72,ANa0xXps,0.131579,0.943396,0.555556,0.534759,0.074975,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,120.45,90.27,0.0,0.0,0.0,0.0,1.068009,0.026974,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9781,2024-01-07,Turquia Superliga,12:00,Bursaspor,Anadolu Efes,1.69,2.12,167.5,1.83,1.91,-3.5,2.01,1.69,67bNM7g8,0.591716,0.471698,0.546448,0.523560,0.063414,0.0,0.0,NaN,NaN,2

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:00,Eua Nba,Brooklyn Nets,Portland Trail Blazers,225.5,1.86,1.0000,Over
1,20:00,Eua Nba,Orlando Magic,Atlanta Hawks,237.5,1.89,1.0000,Over
2,20:00,Eua Nba,Sacramento Kings,New Orleans Pelicans,234.5,1.86,1.0000,Over
3,21:30,Eua Nba,Dallas Mavericks,Minnesota Timberwolves,230.5,1.86,1.0000,Over
4,22:00,Eua Nba,Denver Nuggets,Detroit Pistons,234.5,1.80,1.0000,Over
5,22:00,Eua Nba,Phoenix Suns,Memphis Grizzlies,226.5,1.82,1.0000,Over
6,22:30,Eua Nba,Golden State Warriors,Toronto Raptors,236.5,1.83,0.9999,Over
7,23:30,Eua Nba,Los Angeles Lakers,Los Angeles Clippers,229.5,1.81,1.0000,Over
